# Training script

### Imports

In [6]:
import pandas as pd

from transformers import AutoTokenizer, RobertaModel
from datasets import load_dataset
from torch import nn

### Constants

In [7]:
TRANSFORMER_MODEL_NAME = 'roberta-base'

### Loading the dataset

In [8]:
dataset = load_dataset('multi_woz_v22')

train = pd.DataFrame(columns=dataset['train'].to_pandas().columns)

for _, row in dataset['train'].to_pandas().iterrows():
    if len(row['services']) != 0 and set(row['services']) <= set(['restaurant', 'hotel']):
        train.loc[len(train)] = row
        
print(train.loc[0].turns)

No config specified, defaulting to: multi_woz_v22/v2.2_active_only
Found cached dataset multi_woz_v22 (/home/adrian/.cache/huggingface/datasets/multi_woz_v22/v2.2_active_only/2.2.0/6719c8b21478299411a0c6fdb7137c3ebab2e6425129af831687fb7851c69eb5)


  0%|          | 0/3 [00:00<?, ?it/s]

{'turn_id': array(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11'],
      dtype=object), 'speaker': array([0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]), 'utterance': array(['i need a place to dine in the center thats expensive',
       'I have several options for you; do you prefer African, Asian, or British food?',
       'Any sort of food would be fine, as long as it is a bit expensive. Could I get the phone number for your recommendation?',
       'There is an Afrian place named Bedouin in the centre. How does that sound?',
       'Sounds good, could I get that phone number? Also, could you recommend me an expensive hotel?',
       "Bedouin's phone is 01223367660. As far as hotels go, I recommend the University Arms Hotel in the center of town.",
       'Yes. Can you book it for me?',
       'Sure, when would you like that reservation?',
       'i want to book it for 2 people and 2 nights starting from saturday.',
       'Your booking was successful. Your reference number is F

### Loading model

In [9]:
class TransformerClassifier(nn.Module):
    def __init__(self, base_model_name, num_classes):
        super().__init__()
        self.transformer = RobertaModel.from_pretrained(base_model_name)
        self.pre_classifier = nn.Linear(768, 768)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(768, num_classes)
    
    def forward(self, input):
        cls = self.transformer(**input).last_hidden_state[:, 0, :]
        x = self.pre_classifier(cls)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.classifier(x)
        return x
    

In [10]:
tokenizer = AutoTokenizer.from_pretrained(TRANSFORMER_MODEL_NAME)
classifier = TransformerClassifier(TRANSFORMER_MODEL_NAME, 2)

inputs = tokenizer("We love doing NLI!", return_tensors="pt")
output = classifier(inputs)
print(output.shape)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([1, 2])
